In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os

import sys
## ignore these lines, you do not need to add this if Firefly is pip installed into your PYTHONPATH
sys.path.insert(0, '/Users/ageller/VISUALIZATIONS/Firefly')
sys.path.insert(0,'/Users/agurvich/research/repos/firefly/src')
from firefly.data_reader import FIREreader,SimpleFIREreader,TweenParams
from abg_python.galaxy.gal_utils import Galaxy

don't have phil's colormaps


# Convert FIRE data
In this example notebook we demonstrate how to use the `firefly.data_reader.FIREreader` sub-class which creates specialized data files for FIRE formatted data. The details of how the `FIREreader` class is "specialized" see the <a href="https://alexbgurvi.ch/Firefly/docs/build/html/reference/api/api.html">API documentation</a> and to see the example of this output visit <a href="https://alexbgurvi.ch/firefly/src/firefly/index.html">the live demo version</a>.

In [3]:
galaxy = Galaxy('m12b_res57000',600)
galaxy.extractMainHalo(use_saved_subsnapshots=False)

Couldn't find a metadata file... for
 Metadata object at /Users/agurvich/scratch/data/metal_diffusion/m12b_res57000/metadata/meta_Galaxy_600.hdf5
No rstar 1/2 in halo or metadata files, we will need to calculate it ourselves.
Loading star particles of m12b_res57000 at 600 at /Users/agurvich/snaps/metal_diffusion/m12b_res57000/output
Failed to open saved sub-snapshots AssertionError('Told not to use saved sub-snapshots')
Loading gas particles of m12b_res57000 at 600 at /Users/agurvich/snaps/metal_diffusion/m12b_res57000/output
Loading dark matter particles of m12b_res57000 at 600 at /Users/agurvich/snaps/metal_diffusion/m12b_res57000/output
Reorienting...
Done.
extract_halo_inner(m12b_res57000 at 600,){'orient_stars': True, 'use_saved_subsnapshots': False} 12.79 s elapsed
Snapshot memory free


(12.27201256901771,
 True,
 13.610546783496492,
 -23.753615253930974,
 331.60968,
 2.454402513803542,
 140.56429531755958)

In [4]:
thetas = np.linspace(0,360,1000)/180*np.pi
coords = np.zeros((thetas.size,3))
coords[:,0] = 0
coords[:,1] = np.cos(thetas)
coords[:,2] = np.sin(thetas)
coords*=150
my_tweenParams = TweenParams(coords,30) ## dt = 3000 ms between frames

In [30]:
## create a FIRE reader object
reader = FIREreader(
    ## path to directory containing (optionally multiple) .hdf5 files
    snapdir = "/Users/agurvich/research/snaps/metal_diffusion/m12b_res57000/output/",
    ## the snapshot number, best to provide separately in order to disambiguate
    snapnum = 600,
    ## particle types one would like to extract from .hdf5 files
    ptypes=[0,4,1,2],
    ## what to call them in the UI
    UInames=['Gas','Stars','HRDM','LRDM'],
    ## by what factor would we like to reduce the data for performance stability and disk space concerns
    decimation_factors=[2,2,10,10],
    ## what fields would we like to extract
    fields=['Density','Temperature','AgeGyr','GCRadius'],
    ## do we want to take the magnitude of any of these fields?
    ## do we want to take the log? 
    logFlags=[True,True,False,False],
    ## which fields do we want to be able to filter on?
    filterFlags=[True,True,True,True],
    ## which fields do we want to be able to colormap by?
    colormapFlags=[True,True,True,True],
    radiusFlags=[False,True,True,False],
    ## where should the output .json files be saved to? 
    ##  if a relative path is given, like here, saves to $HOME/<JSONdir>
    ##  and creates a soft-link to firefly/static/data
    datadir=os.path.abspath(os.path.join(os.getcwd(),'..','static','data','FIRESampleData')),
    ## overwrite the existing startup.json file
    write_startup=True,
    clean_datadir=True,
    tweenParams=my_tweenParams)

## fetch data from .hdf5 files
reader.loadData(com=galaxy.scom,vcom=galaxy.sub_snap['vscom'])

Loading ptype 2
Loading ptype 1
Loading ptype 4
Loading ptype 0


array([Gas - 3112865/6225729 particles - 4 tracked fields,
       Stars - 1632362/3264723 particles - 3 tracked fields,
       HRDM - 932304/9323040 particles - 2 tracked fields,
       LRDM - 304227/3042265 particles - 2 tracked fields], dtype=object)

In [34]:
## initialize the camera view
camera = np.array([251,-117,82])
camera = camera/np.linalg.norm(camera)*250
reader.settings['camera'] = camera

## set the initial colors of each of the particle groups
reader.settings['color']['Gas']=[1,0,0,1]
reader.settings['color']['Stars'] = [1,1,1,0.025]
reader.settings['color']['HRDM'] = [0.5,0,0.5,1]
reader.settings['color']['LRDM'] = [0.5,0,0.5,1]

## set the sizes of each of the particle groups
reader.settings['sizeMult']['Gas']=0.25
reader.settings['sizeMult']['Stars']=0.03
reader.settings['sizeMult']['HRDM']=1
reader.settings['sizeMult']['LRDM']=1

In [38]:
## initialize the gas with a colormap
reader.settings['showColormap']['Gas'] = True
reader.settings['blendingMode']['Gas'] = 'normal'
reader.settings['depthTest']['Gas'] = True
reader.settings['colormapVariable']['Gas'] = 3
reader.settings['colormapLims']['Gas']['Velocity'] = [50,500]
reader.settings['colormapVals']['Gas']['Velocity'] = [50,500]

## initialize the gas with radius scaling (index 1 corresponds to Temperature, specified above)
reader.settings['radiusVariable']['Gas'] = 1

## initialize gas with velocity vectors
reader.settings['showVel']['Gas'] = True
reader.settings['velType']['Gas'] = 'arrow'

## initialize HRDM with a filter excluding the center so as not to crowd the view
reader.settings['filterVals']['HRDM']['GCRadius'] = [300,5000]
reader.settings['filterLims']['HRDM']['GCRadius'] = [0,5000]

In [45]:
reader.settings.outputToJSON(reader.datadir,file_prefix=reader.file_prefix)

You will need to add this settings filename to filenames.json if this was not called by a Reader instance.


('/Users/agurvich/research/repos/Firefly/src/firefly/static/data/FIRESampleData/DataSettings.json',
 '/Users/agurvich/research/repos/Firefly/src/firefly/static/data/FIRESampleData/DataSettings.json')

In [36]:
## dump output to .ffly files
reader.writeToDisk()

Gas - 3112865/6225729 particles - 4 tracked fields
Stars - 1632362/3264723 particles - 3 tracked fields
HRDM - 932304/9323040 particles - 2 tracked fields
LRDM - 304227/3042265 particles - 2 tracked fields


''

## Using `firefly.data_reader.SimpleFIREreader`
To simplify this process we have provided `firefly.data_reader.SimpleFIREreader` which takes a path to a FIRE snapshot and creates a basic iteration of Firefly showing the gas and stars given only a path to the data.

In [9]:
reader = SimpleFIREreader(
    ## path to directory containing (optionally multiple) .hdf5 files
    path_to_snapshot = "/Users/agurvich/research/snaps/metal_diffusion/m12b_res57000/output/snapdir_600",
    ## overwrite the existing startup.json file
    write_startup=True, 
    ## pass absolute path to avoid symlink
    JSONdir=os.path.abspath(os.path.join(os.getcwd(),'..','static','data','FIREData_50')))

Loading ptype 4
Loading ptype 0
Outputting: Gas - 622573/6225729 particles - 3 tracked fields
Outputting: Stars - 326473/3264723 particles - 3 tracked fields
